# Solving a Bridge and Torch Problem using Integer Linear Programming (ILP)

# Introduciton

Suppose 4 individuals need to cross a bridge. Its dark though, so they need to take a lantern with them in order to cross. Additionally, the bridge can only support up to 2 people crossing at a time. How should the 4 individuals move across the bridge such that the total time it takes for everyone to the end is finished?

## More Information

For more information about this problem, feel free to check out the Wikipedia page: https://en.wikipedia.org/wiki/Bridge_and_torch_problem.

# Imports

In [1]:
from docplex.mp.model import Model
import pandas as pd

# Input Data

## Sets

$i \in I$: The set of people crossing the bridge

In [2]:
I = ['A', 'B', 'C', 'D']

$t \in T$: The set of time periods



The number of time periods for everyone to cross the bridge:
* 1 Person: 1
* 2 People: 2
* 3 People: 3
* 4 People: 5
* 5 People: 7
* 6 People: 9
* 7 People: 11

Excluding the uninteresting cases of having 2 or fewer people, the number time periods we need is:

$2*len(I)-3$ 

where $len(I)$ is the number of people that need to cross

In [3]:
T = [x for x in range(2*len(I)-2)]

## Parameters

$c_{i}$: The time it takes person $i \in I$ to cross the bridge

In [4]:
c = {
    'A': 1,
    'B': 2,
    'C': 5,
    'D': 8
}

# Optimization Model 

## Create a CPLEX model object

In [5]:
# create a model object
bridge_model = Model(name='bridge+torch')

## Decision Variables

$x_{it}$: $1$ if person $i \in I$ crosses the bridge at time $t \in \{1, ..., |T|\}$ and $0$ otherwise 

In [6]:
x = bridge_model.binary_var_dict(keys = [(i, t) for i in I for t in T[1:]], name="x")

$s_{it}$: $1$ if person $i \in I$ is at the start of the bridge at time $t \in T$ and $0$ otherwise 

In [7]:
s = bridge_model.binary_var_dict(keys = [(i, t) for i in I for t in T], name="s")

$f_{it}$: $1$ if person $i \in I$ is at the end (finish line) of the bridge at time $t \in T$ and $0$ otherwise 

In [8]:
f = bridge_model.binary_var_dict(keys = [(i, t) for i in I for t in T], name="f")

$w_{t}$: The time spent traveling across the bridge at $t \in \{1,..|T|\}$ 

In [9]:
w = bridge_model.continuous_var_dict(keys = [t for t in T], name="w")

## Objective Function

Minimize total walking time:

$minimize \quad \sum_{t = 1}^{|T|} w_{t}$

In [10]:
total_walk_time = bridge_model.sum(w[t] for t in T[1:])
bridge_model.minimize(total_walk_time)

## Constraints

Everyone starts at the beginning

$s_{i0} = 1, \quad \forall i \in I$

In [11]:
bridge_model.add_constraints(s[(i,0)] == 1 for i in I)

[docplex.mp.LinearConstraint[](s_A_0,EQ,1),
 docplex.mp.LinearConstraint[](s_B_0,EQ,1),
 docplex.mp.LinearConstraint[](s_C_0,EQ,1),
 docplex.mp.LinearConstraint[](s_D_0,EQ,1)]

Everyone ends at the end of the bridge/finish line

$f_{i|T|} = 1, \quad \forall i \in I$



In [12]:
bridge_model.add_constraints(f[(i,T[-1])] == 1 for i in I)

[docplex.mp.LinearConstraint[](f_A_5,EQ,1),
 docplex.mp.LinearConstraint[](f_B_5,EQ,1),
 docplex.mp.LinearConstraint[](f_C_5,EQ,1),
 docplex.mp.LinearConstraint[](f_D_5,EQ,1)]

Odd time periods always result in 2 people crossing

$\sum_{i \in I}x_{it} = 2, \quad \forall$ odd $t \in T$

In [13]:
bridge_model.add_constraints(bridge_model.sum(x[(i,t)] for i in I) == 2 for t in T if t % 2 == 1)

[docplex.mp.LinearConstraint[](x_A_1+x_B_1+x_C_1+x_D_1,EQ,2),
 docplex.mp.LinearConstraint[](x_A_3+x_B_3+x_C_3+x_D_3,EQ,2),
 docplex.mp.LinearConstraint[](x_A_5+x_B_5+x_C_5+x_D_5,EQ,2)]

Even time periods always result in 1 person crossing

$\sum_{i \in I}x_{it} = 1, \quad \forall$ even nonzero $t \in T$

In [14]:
bridge_model.add_constraints(bridge_model.sum(x[(i,t)] for i in I) == 1 for t in T[1:] if t % 2 == 0)

[docplex.mp.LinearConstraint[](x_A_2+x_B_2+x_C_2+x_D_2,EQ,1),
 docplex.mp.LinearConstraint[](x_A_4+x_B_4+x_C_4+x_D_4,EQ,1)]

Each person is at the start or end at the end of each time period

$s_{it} + f_{it} = 1, \quad \forall i \in i, \space t \in T$

In [15]:
bridge_model.add_constraints(s[(i,t)] + f[(i,t)] == 1 for i in I for t in T)

[docplex.mp.LinearConstraint[](s_A_0+f_A_0,EQ,1),
 docplex.mp.LinearConstraint[](s_A_1+f_A_1,EQ,1),
 docplex.mp.LinearConstraint[](s_A_2+f_A_2,EQ,1),
 docplex.mp.LinearConstraint[](s_A_3+f_A_3,EQ,1),
 docplex.mp.LinearConstraint[](s_A_4+f_A_4,EQ,1),
 docplex.mp.LinearConstraint[](s_A_5+f_A_5,EQ,1),
 docplex.mp.LinearConstraint[](s_B_0+f_B_0,EQ,1),
 docplex.mp.LinearConstraint[](s_B_1+f_B_1,EQ,1),
 docplex.mp.LinearConstraint[](s_B_2+f_B_2,EQ,1),
 docplex.mp.LinearConstraint[](s_B_3+f_B_3,EQ,1),
 docplex.mp.LinearConstraint[](s_B_4+f_B_4,EQ,1),
 docplex.mp.LinearConstraint[](s_B_5+f_B_5,EQ,1),
 docplex.mp.LinearConstraint[](s_C_0+f_C_0,EQ,1),
 docplex.mp.LinearConstraint[](s_C_1+f_C_1,EQ,1),
 docplex.mp.LinearConstraint[](s_C_2+f_C_2,EQ,1),
 docplex.mp.LinearConstraint[](s_C_3+f_C_3,EQ,1),
 docplex.mp.LinearConstraint[](s_C_4+f_C_4,EQ,1),
 docplex.mp.LinearConstraint[](s_C_5+f_C_5,EQ,1),
 docplex.mp.LinearConstraint[](s_D_0+f_D_0,EQ,1),
 docplex.mp.LinearConstraint[](s_D_1+f_D_1,EQ,1),


The travel time at time $t \in T$ is the travel time of the slower walker

$w_{t} \ge c_{i}x_{it}, \quad \forall i \in I, \space t \in T$

In [16]:
bridge_model.add_constraints(w[t] >= c[i]*x[(i,t)] for i in I for t in T[1:])


[docplex.mp.LinearConstraint[](w_1,GE,x_A_1),
 docplex.mp.LinearConstraint[](w_2,GE,x_A_2),
 docplex.mp.LinearConstraint[](w_3,GE,x_A_3),
 docplex.mp.LinearConstraint[](w_4,GE,x_A_4),
 docplex.mp.LinearConstraint[](w_5,GE,x_A_5),
 docplex.mp.LinearConstraint[](w_1,GE,2x_B_1),
 docplex.mp.LinearConstraint[](w_2,GE,2x_B_2),
 docplex.mp.LinearConstraint[](w_3,GE,2x_B_3),
 docplex.mp.LinearConstraint[](w_4,GE,2x_B_4),
 docplex.mp.LinearConstraint[](w_5,GE,2x_B_5),
 docplex.mp.LinearConstraint[](w_1,GE,5x_C_1),
 docplex.mp.LinearConstraint[](w_2,GE,5x_C_2),
 docplex.mp.LinearConstraint[](w_3,GE,5x_C_3),
 docplex.mp.LinearConstraint[](w_4,GE,5x_C_4),
 docplex.mp.LinearConstraint[](w_5,GE,5x_C_5),
 docplex.mp.LinearConstraint[](w_1,GE,8x_D_1),
 docplex.mp.LinearConstraint[](w_2,GE,8x_D_2),
 docplex.mp.LinearConstraint[](w_3,GE,8x_D_3),
 docplex.mp.LinearConstraint[](w_4,GE,8x_D_4),
 docplex.mp.LinearConstraint[](w_5,GE,8x_D_5)]

Logic for connecting x variables to f and p variables

$f_{p(t-1)} + x_{pt} = f_{pt}, \quad \forall$ odd $t \in T$

$s_{p(t-1)} + x_{pt} = s_{pt}, \quad \forall$ nonzero even $t \in T$

In [17]:
bridge_model.add_constraints(f[(i,t-1)] + x[(i,t)] == f[(i,t)] for i in I for t in T if t % 2 == 1)
bridge_model.add_constraints(s[(i,t-1)] + x[(i,t)] == s[(i,t)] for i in I for t in T[1:] if t % 2 == 0)

[docplex.mp.LinearConstraint[](x_A_2+s_A_1,EQ,s_A_2),
 docplex.mp.LinearConstraint[](x_A_4+s_A_3,EQ,s_A_4),
 docplex.mp.LinearConstraint[](x_B_2+s_B_1,EQ,s_B_2),
 docplex.mp.LinearConstraint[](x_B_4+s_B_3,EQ,s_B_4),
 docplex.mp.LinearConstraint[](x_C_2+s_C_1,EQ,s_C_2),
 docplex.mp.LinearConstraint[](x_C_4+s_C_3,EQ,s_C_4),
 docplex.mp.LinearConstraint[](x_D_2+s_D_1,EQ,s_D_2),
 docplex.mp.LinearConstraint[](x_D_4+s_D_3,EQ,s_D_4)]

# Solve the problem

In [18]:
bridge_model.solve()

docplex.mp.solution.SolveSolution(obj=15,values={x_A_1:1,x_A_4:1,x_A_5:1..

## Objective Function Value

In [19]:
bridge_model.solution.objective_value

15.0

## Who crosses at each time

In [20]:
x_values = bridge_model.solution.get_value_df(x)
x_values.loc[x_values.value == 1].rename(columns={"key_1":"Person","key_2":"Time Period"})[["Time Period","Person"]].sort_values('Time Period')

,Time Period,Person
0,1,A
5,1,B
6,2,B
12,3,C
17,3,D
3,4,A
4,5,A
9,5,B


## Send Results to JSON

In [21]:
bridge_model.solution.export("bridge_and_torch_solution.json")